In [2]:
import pandas as pd 
final = pd.read_csv("final.csv")

from sklearn.preprocessing import StandardScaler

# normalizing the features which ensures that they are on the same scale 
scaler = StandardScaler()
df_scaled = scaler.fit_transform(final[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']])




In [3]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(df_scaled)

def get_song_recommendations(song_name, top_n=3):
    
    # filter the dataframe 
    song_df = final[final['Track name'] == song_name]
    
  
    if song_df.empty:
        raise ValueError(f"Song '{song_name}' not found in the dataset.")
    
   
    song_index = song_df.index[0]
    
    # calculate cosine similarity
    similarity_scores = list(enumerate(similarity_matrix[song_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:top_n+1]  # Exclude the input song itself
    song_indices = [i[0] for i in similarity_scores]
    
    return final.iloc[song_indices]

# Attempt to see if it works 
try:
    recommendations = get_song_recommendations('Love Story')
    print(recommendations)
except ValueError as e:
    print(e)


                    Track ID                     Track name   Artist name  Popularity score  Release year Genre  danceability  energy  key  loudness  mode  speechiness  acousticness  instrumentalness  liveness  valence    tempo  time_signature
1476  0kN3oXYWWAk1uC0y2WoyOE                     Love Story  Taylor Swift                47          2008  Love         0.618   0.736    2    -3.937     1       0.0308         0.157          0.000000    0.0730    0.307  118.982               4
1436  3CeCwYWvdfXbZLXFhBrbnf  Love Story (Taylor’s Version)  Taylor Swift                78          2021  Love         0.626   0.790    2    -4.302     1       0.0311         0.135          0.000004    0.0989    0.416  119.082               4
391   1V2ND6AWTXlzCnZ1s2klMH               Glow In The Dark   Tom Gregory                53          2024  GLOW         0.669   0.714    2    -5.208     1       0.0452         0.112          0.000000    0.0870    0.400  128.030               4


In [4]:
get_song_recommendations("Friday")



,Track ID,Track name,Artist name,Popularity score,Release year,Genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1436,3CeCwYWvdfXbZLXFhBrbnf,Love Story (Taylor’s Version),Taylor Swift,78,2021,Love,0.626,0.790,2,-4.302,1,0.0311,0.1350,0.000004,0.0989,0.416,119.082,4
55,5xR8ngEQmlQ1U2xSE0KzTo,BMXXing,mgk,68,2024,New Releases,0.746,0.839,1,-5.534,1,0.0599,0.0429,0.000016,0.1470,0.462,105.059,4
258,3lOMJTQTd6J34faYwASc33,POP!,NAYEON,73,2022,Pop,0.795,0.859,2,-2.994,1,0.0542,0.0361,0.000000,0.0321,0.356,96.986,4


In [8]:
#mood based recommendations 

mood_dict = {
    'happy': {'valence': (0.7, 1.0), 'danceability': (0.6, 1.0), 'energy': (0.6, 1.0)},
    'sad': {'valence': (0.0, 0.4), 'danceability': (0.0, 0.4), 'energy': (0.0, 0.4)},
    'relaxed': {'valence': (0.4, 0.7), 'danceability': (0.4, 0.7), 'energy': (0.4, 0.7)},
    'energetic': {'valence': (0.6, 1.0), 'danceability': (0.7, 1.0), 'energy': (0.7, 1.0)}
}

# adjust pandas display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

def get_mood_recommendations(mood, top_n=3):
    if mood not in mood_dict:
        raise ValueError(f"Mood '{mood}' not found in mood_dict")
    
    mood_features = mood_dict[mood]
    
   
    filtered_songs = final
    for feature, (low, high) in mood_features.items():
        filtered_songs = filtered_songs[(final[feature] >= low) & (final[feature] <= high)]
    
   
    if len(filtered_songs) < top_n:
        top_songs = filtered_songs
    else:
        
        top_songs = filtered_songs.nlargest(top_n, 'Popularity score')
    
   
    top_songs = top_songs[['Track name', 'Artist name']]
    
    
    print(top_songs)

# Example: 
get_mood_recommendations('sad')


                                             Track name             Artist name
1460                               lovely (with Khalid)           Billie Eilish
1269  White Noise Sleeping Aid to Help My Baby Fall ...  White Noise Baby Sleep
746                                             Forever              Noah Kahan


C:\Users\harip\AppData\Local\Temp\ipykernel_19412\1728261240.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_songs = filtered_songs[(final[feature] >= low) & (final[feature] <= high)]
C:\Users\harip\AppData\Local\Temp\ipykernel_19412\1728261240.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_songs = filtered_songs[(final[feature] >= low) & (final[feature] <= high)]
